In [ ]:
## Load necessary libraries
%matplotlib inline
from constructIDF import *
import pandas as pd
import numpy as np
import itertools
import argparse
import matplotlib.pyplot as plt

from matplotlib import rcParams

rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14

### Step 1: Read file with daily rainfall from downscaled GCM ouput.

#### Step 1.1: Specify path to hourly rainfall time series

Specify the path to where you downloaded the historical downscaled GCM data in the cell below, between quotation marks. The standard path looks like: "/users/user/documents/cmu/rainfall/maca_hist.csv"

In [ ]:
"[*]"

hist_gcm_path = ""
hist_gcm_data = pd.read_csv(historical_path, skiprows=26, parse_dates=["yyyy-mm-dd"])

### Step 2: Construct Annual Maximum Series

#### Step 2.1: Reformat data to functions' required input format
We need to reformat the data to the required format by the library `constructIDF`

In [ ]:
hist_gcm_data['year'] = hist_gcm_data["yyyy-mm-dd"].dt.year

#### Step 2.2: Extract Annual Maximum Series

hist_gcm_ams is the variable where we store the annual maxima (the line of code in the following cell extracts the maximum daily rainfall each year over the historical period)

In [ ]:
hist_gcm_ams = hist_gcm_data.groupby(pd.Grouper(key="yyyy-mm-dd", freq='A')).max()

cols = list(hist_gcm_ams)
cols.insert(0, cols.pop(cols.index('year')))
hist_gcm_ams = hist_gcm_ams.loc[:, cols]

hist_gcm_ams = hist_gcm_ams.reset_index(drop=True)

#### Step 2.3: Take a look at the AMS output table

The table below correspond to the annual maxima each year (row) and downscaled GCM (column) from MACA.

In [ ]:

#########################
#                       # 
#   Historical Model    #
#      AMS TABLE        #
#                       #
#########################

hist_gcm_ams

### Step 3: Fit Generalized Extreme Value and obtain precipitation estimates (inches or mm) by average recurrence interval (ARI)

The next step is to fit a generalized extreme value distribution to each duration's AMS. Once the parameters (location, scale and shape) are estimated, these are used to retrieve the return levels (in this case, rainfall depth) for different quantiles feed into the inverse of the CDF. Usually, the quantiles are equal to the inverse of the average recurrence interval (ARI) (e.g. 1/2 = 2-year).

`constructIDF` has one method that merges all these steps, but we need to specify if we want to construct confidence intervals. The method implemented in `constructIDF` is bootstrapping, so we also need to specify the number of bootsrapped samples. Default value is 1000, and using a smaller number is not recommended.

Other specification is the confidence level, alpha, used to estimate the confidence intervals. Default is 0.9 (90% confidence interval).


In this tutorial, we will compute confidence intervals at a 90% confidence level using 1000 bootstrapped samples.

```python
ci = True
alpha = 0.9
number_bootstrap = 1000
```

#### Step 3.1: Specify values for IDF function and run the function

In [ ]:
ci = True
alpha = 0.9
number_bootstrap = 100

hist_gcm_idf = IDF(hist_gcm_ams, ci, number_bootstrap, alpha)

#### Step 3.2: Construct IDF from the function IDF output.

Note: This will take long time because of the number of bootstrapped samples. The constructed IDF is by default for the following ARI: 2-, 5-, 10-, 25-, 50-, 100-, 200-year. Some errors will be displayed, no worries. 


In [ ]:
hist_gcm_idf.construct_IDF()

#### Step 3.3: Access the IDF table

We can access the table with the precipitation estimate values
for each return period and the corresponding 90% CI confidence bounds. 

In [ ]:
######################################
#                                    # 
#             IDF TABLE              #
#       Historical (1950-2005)       #
#                                    #
######################################


hist_gcm_idf.idf

### Step 4: Repeat the process, but now using future data as input

In [ ]:
"[*]"

"""
Specify here the path to the precipitation data corresponding to the future period.
"""
fut_gcm_path = ""
fut_gcm_data = pd.read_csv(fut_gcm_path, skiprows=26, parse_dates=["yyyy-mm-dd"])

#### Step 4.1: Reformat data 

In [ ]:
fut_gcm_data['year'] = fut_gcm_data["yyyy-mm-dd"].dt.year
fut_gcm_ams = fut_gcm_data.groupby(pd.Grouper(key="yyyy-mm-dd", freq='A')).max()



cols = list(fut_gcm_ams)
cols.insert(0, cols.pop(cols.index('year')))
fut_gcm_ams = fut_gcm_ams.loc[:, cols]

#### Step 4.2: Filter the AMS series to select a period that matches the historical period length. In other words, if we have 56 years of annual maxima in the period 1950-2005, we need to select a future period that spans the same number of years. In this case, 2043-2099.



In [ ]:
fut_gcm_ams = fut_gcm_ams['2043':'2099']
fut_gcm_ams = fut_gcm_ams.reset_index(drop=True)

In [ ]:
#########################
#                       #
#      Future Model     #
#       AMS TABLE       #
#                       #
#########################

fut_gcm_ams

#### Step 4.3: Input the future AMS and the same specifications when we calculated the historical gcm IDF values.

In [ ]:
fut_gcm_idf = IDF(fut_gcm_ams, ci, number_bootstrap, alpha)

#### Step 4.4: Construct IDF from the data we feed above and our specifications. 

Some errors will be displayed, no worries. This will take long time because of the number of bootsrapped samples. The constructed IDF is by default for the following ARI: 2-, 5-, 10-, 25-, 50-, 100-, 200-year

In [ ]:
fut_gcm_idf.construct_IDF()

#### Step 4.5: Take a look at the future GCM idf values.

**Note** that we no longer have *durations* by column. 
We are focusing on the 24h. Now, each column corresponds to the 24h future idf GCM precipitation estimate for each return period and confidence bound.

In [ ]:
######################################
#                                    # 
#          Future Model              #
#           IDF TABLE                #
#                                    #
######################################

fut_gcm_idf.idf

### Step 5: Find the precipitation estimate change factor by return period using the delta-change method

The change factor is equal to the ratio between historical IDF values and future IDF values (both estimated using the downscaled GCM data which is what we have been doing in the previous steps).

In [ ]:
change_factors = pd.DataFrame(fut_gcm_idf.idf.values/hist_gcm_idf.idf.values)

# Reformat table for better presentation
change_factors.columns = [x.rstrip("_rcp85(mm)") for x in fut_gcm_idf.idf.columns]
change_factors['return_period'] = fut_gcm_idf.idf.index
change_factors.set_index('return_period', inplace=True)

In [ ]:
####################################
#                                  # 
#          CHANGE FACTORS          #
#             TABLE                #
#                                  #
####################################

change_factors

### Step 6: Update historical (station) IDF curves using change factor estimated in previous steps

We can construct a future IDF by updating the historical curve using the change factor that we estimated from the downscaled projections. 

Note that there are two options here:

1. Assume equal change in all storm durations to the estimated change in the 24h rainfall depth (the change factors we computed above). 
2. Update the 24h only as we computed a change factor using daily downscaled GCM output

In this exercise, we will choose option 2


#### Step 6.1: Load idf values (CSV) from station IDF curve (24h)

Insert the path to your data between the quotation marks in the line below.

In [ ]:
"[*]"

hist_station_idf = pd.read_csv("", index_col=0)

#### Step 6.2: Multiply the station 24h IDF curve (`hist_station_idf`) by change factor table (`change_factors`)

The code below is multiplying the station 24h curve values
with the change factors table computed in the previous steps.

In [ ]:
######################################
#                                    # 
#         Station Future             #
#           IDF TABLE                #
#                                    #
######################################

fut_station_idf = change_factors.multiply(hist_station_idf["24H"], axis="index")

#### Step 6.3: Take a look at the updated station IDF values (now co)

In [ ]:
fut_station_idf

### Step 7: Save the table above and plot updated station IDF curves


#### Step 7.1: Store table in CSV or compute statistics in this notebook

The `fut_station_idf` correspond to the station future 24h precipitation estimate projected by each GCM.
You can perform ensemble statistics (take the mean, median or quantiles across the GCMs) to get a sense of
what the future 24h precipitation value at the station will be given the projected values from the GCMs.

You can save the table stored in the variable `fut_station_idf` to a CSV using pandas. The line of code will look like (you need to replace the `path/to/csv` portion to your desired save location):

`fut_station_idf.to_csv("path/to/csv/fut_station_idf.csv")` 

#### Step 7.2: Plot idf curves for each GCM model

We can call the `plot_IDF` method to create the IDF curves and plot them.
We need to specify a path where to store the figure, as well as the figure format.

Specify in the cell below:

In [ ]:
"[*]"

figure_path = "."
figure_name = "fut_station_idf_plot"
figure_format = 'png'

In [ ]:
station_idf_transposed = fut_station_idf.transpose()
dfmean = station_idf_transposed.drop([x for x in station_idf_transposed.columns if (
                x[:1] == 'L' or x[:1] == 'U')], axis=1)

dfmean = dfmean.transpose()
fig, axs = plt.subplots(figsize=(13, 10))
a1 = dfmean.plot(ax=axs)
fill_alpha = 0.3

legend = plt.legend(title='GCM Model', fontsize=13)
plt.setp(legend.get_title(), fontsize=15)
plt.ylabel('Precipitation Depth (mm)', {'fontsize': 18})
plt.xlabel('Return Period', {'fontsize': 18})
plt.grid()

plt.savefig("{}/{}.{}".format(figure_path, figure_name, figure_format), bbox_inches='tight')